learning objectives:
Data Pipeline as an efficient methodology

why it matters:
for experimentation purpose. 

in this phase, we will 
collect, 
LOAD,
clean (missing values, duplicate), 
splitting (train and test), 
and validate data.


in this way, we will have an efficient steps.
kalau pengen pakaid dataset lain, udah enak 


In [1]:
! pip install pandas seaborn scikit-learn imblearn pyyaml kaggle


In [2]:
import os

os.environ['KAGGLE_USERNAME'] = '--kaggle_username--'
os.environ['KAGGLE_KEY'] = '---key from kaggle---'


In [ ]:
import pandas as pd
import src.utils as utils
import os


from sklearn.model_selection import train_test_split

# Load Configuration File

test markdown

In [ ]:
CONFIG_DATA = utils.config_load()
CONFIG_DATA

In [ ]:
CONFIG_DATA['raw_dataset_path']

## Download using Kaggle API


In [ ]:
pip install kaggle

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile("./creditcardfraud.zip", 'r') as file:
    file.extractall(
        path="./data/raw")

In [ ]:
os.rename("data/raw/creditcard.csv", CONFIG_DATA['raw_dataset_path'])

# Data Collection

1. Membaca file raw 
2. Split data menjadi Train, Valid, Test
3. Simpan data Train, Valid dan Test 

In [ ]:
#baca data CSV, simpan sebagai pickle
CONFIG_DATA['raw_dataset_path']

In [ ]:
'''
baca csv, simpan sebagai pickle. Format data yang lebih efisien. misal: csv 1 Gb jadi 0.5Gb. 
seringnya sih dari  database. data parquet lebih ok lagi sih; Pandas3 is optimized using parquet. more efficient
'''

CONFIG_DATA['data_set_path']

In [ ]:
def read_data(return_file=True):
    # Read data
    data = pd.read_csv(CONFIG_DATA['raw_dataset_path'], 
                       sep=',',
                    #    index_col=CONFIG_DATA['index_column'])
    )
    # Print data
    print('data shape   :', data.shape)

    # Dump data
    utils.pickle_dump(data, CONFIG_DATA['data_set_path'])

    # Return data
    if return_file:
        return data

In [ ]:
data = read_data()
data.head()

In [ ]:
data.describe()

# Data Splitting

X = features <br>
y = label

waktu training perlu ada 2 input ke algoritma <br>
1. feature
2. label

In [ ]:
def split_input_output(return_file=True):
    # Read data
    data = utils.pickle_load(CONFIG_DATA['data_set_path'])

    # Split input & output
    y = data[CONFIG_DATA['output_column']]
    X = data.drop([CONFIG_DATA['output_column']], axis=1)

    # Print splitting
    print('Input shape  :', X.shape)
    print('Output shape :', y.shape)
    print('Input NAN    :')
    print(X.isnull().sum())
    print('Benchmark    :')
    print(y.value_counts(normalize=True))
    
    # Dump file
    utils.pickle_dump(X, CONFIG_DATA['input_set_path'])
    utils.pickle_dump(y, CONFIG_DATA['output_set_path'])
    utils.pickle_dump(X.columns, CONFIG_DATA['input_columns_path'])     # dump input columns

    if return_file:
        return X, y

In [ ]:
X, y = split_input_output()

In [ ]:
CONFIG_DATA['input_set_path']

In [ ]:
CONFIG_DATA['test_size']

train = melatih model <br>
valid = optimize model <br>
test = memilih model yang paling baik

model 1 
- training = 0.85
- valid = 0.9

model 2
- training = 0.83
- valid = 0.88

menggunakan data test untuk memilih model paling bagus nilainya <br>

model 1
- test = 0.75
model 2
- test = 0.8

kesimpulan : kita akan menggunakan model 2 sebagi model akhir

data raw 200.000 <br>
data test = 20 % = 40.000 <br>
data valid = 160.000 x 20% = 32.000 <br>
data train = 200.000 - 40.000 - 32.000 = 138.000

persentase test 10 - 25 %

In [ ]:
def split_train_test(return_file=True):
    # Load data
    X = utils.pickle_load(CONFIG_DATA['input_set_path'])
    y = utils.pickle_load(CONFIG_DATA['output_set_path'])

    # Split test & rest (train & valid)
    X_train, X_test, y_train, y_test = train_test_split( 
                                            X,
                                            y,
                                            test_size = CONFIG_DATA['test_size'],
                                            random_state = CONFIG_DATA['seed']
                                        )
    
    # Split train & valid
    X_train, X_valid, y_train, y_valid = train_test_split(
                                            X_train,
                                            y_train,
                                            test_size = CONFIG_DATA['test_size'],
                                            random_state = CONFIG_DATA['seed']
                                        )
    
    # Print splitting
    print('X_train shape :', X_train.shape)
    print('y_train shape :', y_train.shape)
    print('X_valid shape  :', X_valid.shape)
    print('y_valid shape  :', y_valid.shape)
    print('X_test shape  :', X_test.shape)
    print('y_test shape  :', y_test.shape)

    # Dump file
    utils.pickle_dump(X_train, CONFIG_DATA['train_set_path'][0])
    utils.pickle_dump(y_train, CONFIG_DATA['train_set_path'][1])
    utils.pickle_dump(X_valid, CONFIG_DATA['valid_set_path'][0])
    utils.pickle_dump(y_valid, CONFIG_DATA['valid_set_path'][1])
    utils.pickle_dump(X_test, CONFIG_DATA['test_set_path'][0])
    utils.pickle_dump(y_test, CONFIG_DATA['test_set_path'][1])

    if return_file:
        return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test = split_train_test()

Get sample for testing

In [ ]:
import numpy as np

In [ ]:
np.random.seed(123)
y_sample_0 = y_test[y_test==0].sample(10)
y_sample_1 = y_test[y_test==1].sample(10)

y_sample = pd.concat((y_sample_0, y_sample_1), axis=0)
y_sample

In [ ]:
X_sample = X_test.loc[y_sample.index]
X_sample

In [ ]:
X_sample.to_csv('data/output/X_sample.csv', index=False)

In [ ]:
y_sample.to_csv('data/output/y_sample.csv', index=False)